In [1]:
#pip install selenium
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.chrome.service import Service

In [2]:
#pip install --upgrade selenium

In [3]:
chrome_options = webdriver.ChromeOptions()
# Add any additional options you need

service = Service(executable_path="C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")

# Specify the binary location (replace with your Chrome executable path)
chrome_options.binary_location = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe"

# Launch the driver
#driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome()

In [4]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
MAD
Where to?
DUB
----------
From which city?
DUB
Where to?
MAD
----------
From which city?
-1

Routes:
MAD => DUB
DUB => MAD


In [5]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2024-03-01
End Date, Please use YYYY-MM-DD format only 2024-03-03


In [6]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('div',class_='J0g6-operator-text',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('span',class_='JWEO-stops-text')

    for i in stops:
        stops_list.append(i.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='f8F1-price-text')

    for i in price:
        prices.append(i.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='xdW8 xdW8-mod-full-airport')
    for i in duration:
        duration_list.append(i.text)
    return duration_list

In [7]:
# Set the path to the ChromeDriver executable
chromedriver_path = "C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Launch the driver using the Service class
service = Service(chromedriver_path)
#driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome()

for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination", "Duration", "Total stops", "Price", "Date"]
    df = pd.DataFrame(columns=column_names)

    for j in tqdm(range(num_days + 1)):
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            #service = Service(chromedriver_path)
            #driver = webdriver.Chrome(service=service)
            service = Service()
            driver = webdriver.Chrome()

        url = f"https://www.kayak.ie/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)

        # click show more button to get all flights
        try:
            show_more_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="ULvh-button show-more-button"]'))
            )
            show_more_button.click()
            print("Button pressed")
        except NoSuchElementException:
            print("Show more button not found. Skipping to the next iteration.")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        print(airlines)
        print(total_stops)
        print(prices)
        print(duration)
        new_data = pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops': total_stops,
            'Price': prices,
            'Date': start_date + j
        })
        df = pd.concat([df, new_data], ignore_index=True, axis=0)

    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n', '', regex=True)
    df = df.reset_index(drop=True)

    # save data as a CSV file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv', index=False)
    print(f"Successfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv")

driver.quit()

  0%|          | 0/3 [00:00<?, ?it/s]

Button pressed


 33%|███▎      | 1/3 [00:26<00:52, 26.13s/it]

['Aer Lingus', 'Aer Lingus', 'Aer Lingus', 'Iberia', 'Iberia', 'British Airways', 'British Airways', 'Aer Lingus', 'Iberia', 'Iberia Express', 'Vueling', 'British Airways', 'Transavia France, Vueling', 'TAP AIR PORTUGAL', 'Vueling', 'Transavia France']
['direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', '1 stop', '1 stop', '1 stop', '1 stop']
['€45', '€39', '€39', '€65', '€65', '€103', '€103', '€126', '€139', '€140', '€147', '€181', '€93', '€91', '€98', '€109']
['2h 40m', '2h 40m', '2h 45m', '2h 40m', '2h 45m', '2h 40m', '2h 45m', '2h 40m', '2h 40m', '2h 40m', '2h 40m', '2h 40m', '5h 05m', '6h 45m', '6h 15m', '5h 15m']
Button pressed


 67%|██████▋   | 2/3 [00:43<00:20, 20.99s/it]

...

['Aer Lingus', 'Iberia', 'Iberia Express', 'Aer Lingus', 'Vueling', 'Aer Lingus', 'Aer Lingus', 'Iberia', 'KLM', 'KLM', 'KLM', 'Iberia', 'TAP AIR PORTUGAL', 'KLM', 'Vueling', 'British Airways']
['direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', '1 stop', '1 stop', '1 stop', 'direct', '1 stop', '1 stop', '1 stop', '1 stop']
['€90', '€67', '€78', '€90', '€98', '€150', '€170', '€202', '€108', '€108', '€108', '€236', '€105', '€108', '€123', '€136']
['2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 50m', '2h 35m', '5h 05m', '5h 10m', '5h 20m', '2h 50m', '5h 50m', '6h 15m', '5h 25m', '5h 05m']
Button pressed


 67%|██████▋   | 2/3 [00:43<00:21, 21.17s/it]

['Aer Lingus', 'Aer Lingus', 'British Airways, easyJet', 'Iberia', 'Iberia Express', 'Iberia', 'Aer Lingus', 'Vueling', 'KLM', 'British Airways', 'British Airways', 'British Airways', 'British Airways', 'KLM', 'British Airways', 'TAP AIR PORTUGAL', 'Aer Lingus, easyJet']
['direct', 'direct', '1 stop', 'direct', 'direct', 'direct', 'direct', 'direct', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€78', '€72', '€65', '€90', '€98', '€102', '€117', '€128', '€96', '€100', '€104', '€110', '€100', '€109', '€118', '€106', '€94']
['2h 50m', '2h 50m', '19h 50m', '2h 50m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '5h 10m', '5h 35m', '5h 30m', '5h 10m', '6h 10m', '5h 20m', '5h 15m', '5h 50m', '6h 10m']
Button pressed


100%|██████████| 3/3 [01:00<00:00, 20.29s/it]

['Aer Lingus', 'Aer Lingus', 'Iberia', 'Iberia', 'Iberia Express', 'Aer Lingus', 'Vueling', 'British Airways', 'KLM', 'British Airways', 'KLM', 'KLM', 'TAP AIR PORTUGAL', 'KLM', 'Aer Lingus, Iberia', 'Aer Lingus, Vueling']
['direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', 'direct', '1 stop', 'direct', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€78', '€72', '€131', '€149', '€149', '€157', '€167', '€172', '€97', '€215', '€110', '€110', '€106', '€110', '€101', '€106']
['2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '5h 20m', '2h 35m', '5h 05m', '5h 10m', '5h 50m', '6h 15m', '5h 55m', '5h 55m']
Successfully saved DUB => MAD route as DUB_MAD.csv
